In [23]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
print(spark)
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [11]:
df1 = spark.read.csv("pune_home_prices.csv",header='true',inferSchema='true')
df1.show(5)

+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
|           area_type| availability|            location|     size|society|total_sqft|bath|balcony|price|
+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
|Super built-up  Area|       19-Dec|Electronic City P...|    2 BHK|Coomee |      1056|   2|      1|39.07|
|          Plot  Area|Ready To Move|    Chikka Tirupathi|4 Bedroom|Theanmp|      2600|   5|      3|120.0|
|      Built-up  Area|Ready To Move|         Uttarahalli|    3 BHK|   null|      1440|   2|      3| 62.0|
|Super built-up  Area|Ready To Move|  Lingadheeranahalli|    3 BHK|Soiewre|      1521|   3|      1| 95.0|
|Super built-up  Area|Ready To Move|            Kothanur|    2 BHK|   null|      1200|   2|      1| 51.0|
+--------------------+-------------+--------------------+---------+-------+----------+----+-------+-----+
only showing top 5 rows



In [21]:
print("(",df1.count(),",", len(df1.columns),")")
df1.printSchema()

( 13320 , 9 )
root
 |-- area_type: string (nullable = true)
 |-- availability: string (nullable = true)
 |-- location: string (nullable = true)
 |-- size: string (nullable = true)
 |-- society: string (nullable = true)
 |-- total_sqft: string (nullable = true)
 |-- bath: integer (nullable = true)
 |-- balcony: integer (nullable = true)
 |-- price: double (nullable = true)



In [24]:
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

In [25]:
df1.shape()

(13320, 9)

In [28]:
df1.select('area_type','availability','location').show(5)

+--------------------+-------------+--------------------+
|           area_type| availability|            location|
+--------------------+-------------+--------------------+
|Super built-up  Area|       19-Dec|Electronic City P...|
|          Plot  Area|Ready To Move|    Chikka Tirupathi|
|      Built-up  Area|Ready To Move|         Uttarahalli|
|Super built-up  Area|Ready To Move|  Lingadheeranahalli|
|Super built-up  Area|Ready To Move|            Kothanur|
+--------------------+-------------+--------------------+
only showing top 5 rows



In [31]:
df1.select('area_type').distinct().show()

+--------------------+
|           area_type|
+--------------------+
|      Built-up  Area|
|Super built-up  Area|
|          Plot  Area|
|        Carpet  Area|
+--------------------+



In [37]:
df1.groupBy('area_type').count().show()

+--------------------+-----+
|           area_type|count|
+--------------------+-----+
|      Built-up  Area| 2418|
|Super built-up  Area| 8790|
|          Plot  Area| 2025|
|        Carpet  Area|   87|
+--------------------+-----+



In [38]:
df1.groupBy('balcony').count().show()

+-------+-----+
|balcony|count|
+-------+-----+
|   null|  609|
|      1| 4897|
|      3| 1672|
|      2| 5113|
|      0| 1029|
+-------+-----+



In [40]:
df2 = df1.select('location','size','total_sqft','bath','price')
df2.shape()

(13320, 5)

In [41]:
df2.show(5)

+--------------------+---------+----------+----+-----+
|            location|     size|total_sqft|bath|price|
+--------------------+---------+----------+----+-----+
|Electronic City P...|    2 BHK|      1056|   2|39.07|
|    Chikka Tirupathi|4 Bedroom|      2600|   5|120.0|
|         Uttarahalli|    3 BHK|      1440|   2| 62.0|
|  Lingadheeranahalli|    3 BHK|      1521|   3| 95.0|
|            Kothanur|    2 BHK|      1200|   2| 51.0|
+--------------------+---------+----------+----+-----+
only showing top 5 rows



In [52]:
df2.filter("location is null").show()

+--------+-----+----------+----+-----+
|location| size|total_sqft|bath|price|
+--------+-----+----------+----+-----+
|    null|3 BHK|      1600|   3| 86.0|
+--------+-----+----------+----+-----+



In [53]:
df2.filter("size is null").show(5)

+--------------+----+-----------+----+------+
|      location|size| total_sqft|bath| price|
+--------------+----+-----------+----+------+
|Sarjapur  Road|null|1200 - 2400|null|34.185|
|      IVC Road|null|2000 - 5634|null| 124.0|
|  Banashankari|null|       2400|null| 460.0|
|Sarjapur  Road|null|1200 - 2400|null|28.785|
|   Devanahalli|null|1500 - 2400|null|  46.8|
+--------------+----+-----------+----+------+
only showing top 5 rows



In [58]:
df3 = df2.dropna(thresh=2,subset=('location','size','bath'))  

df3.show(5)

+--------------------+---------+----------+----+-----+
|            location|     size|total_sqft|bath|price|
+--------------------+---------+----------+----+-----+
|Electronic City P...|    2 BHK|      1056|   2|39.07|
|    Chikka Tirupathi|4 Bedroom|      2600|   5|120.0|
|         Uttarahalli|    3 BHK|      1440|   2| 62.0|
|  Lingadheeranahalli|    3 BHK|      1521|   3| 95.0|
|            Kothanur|    2 BHK|      1200|   2| 51.0|
+--------------------+---------+----------+----+-----+
only showing top 5 rows



In [59]:
df3.shape()

(13304, 5)